In [1]:
import pandas as pd
from datetime import datetime
# import datetime
import re
import requests
from requests import get
from bs4 import BeautifulSoup
import warnings
from time import sleep
from random import randint
from time import time
timestart_time = time()
warnings.warn('Warning Simulation')
from IPython.core.display import clear_output

C:\Users\k.kiki\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Warning Simulation
  del sys.path[0]


In [2]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
raw_html = requests.get('https://www.moh.gov.gr/articles/ministry/grafeio-typoy/press-releases')
soup = BeautifulSoup(raw_html.content, "html.parser") 
articles_container = soup.find('div',{'class':'col-lg-7 col-md-12 px-3 main_content'})
titles = articles_container.find_all('a', href=True)

In [4]:
raw_html = requests.get('https://www.moh.gov.gr/articles/ministry/grafeio-typoy/press-releases')
# raw_html
soup = BeautifulSoup(raw_html.content, "html.parser") 
articles_container = soup.find('div',{'class':'col-lg-7 col-md-12 px-3 main_content'})
# articles_container
titles = articles_container.find_all('a', href=True)
# titles

frame = []

for t in titles:
    entries = {}
    
    if 'νακοίνωση για την εξέλιξη της νόσου COVID-19 στη χώρα μας' in t.text\
        or 'νημέρωση διαπιστευμένων συντακτών υγείας από' in t.text:

        entries['title'] = t.text
        entries['url'] = t.get('href')

        frame.append(entries)
    
frame = pd.DataFrame(frame)
frame['date'] = frame.title.str.extract(r'[(](\d+\/\d+).*[)]')
frame['date'] = frame.date+'/2020'
frame['date'] = pd.to_datetime(frame['date'])
frame['date'] = pd.to_datetime(frame["date"], format="%Y-%m-%d").dt.strftime("%Y-%m-%d")

In [5]:
def get_press_release(row):
    page = requests.get(row['url'])
    soup_doc = BeautifulSoup(page.content, "html.parser") 
    
    par = ''
    for p in soup_doc.find_all('p'):
        par = par+p.text.strip()
        
    return pd.Series({
    'content': par 
    }) 

In [6]:
df = frame.apply(get_press_release, axis=1).join(frame)
# df.head()

In [7]:
df['new_cases'] = df.content.str.extract(r'.*[aα]νακοινώνο\w+\s+(\w+)\s+νέα\s+κρούσματα',re.IGNORECASE)

In [8]:
df['new_cases'] = df.new_cases.replace({'μηδέν':'0',
                                          'κανέναν':'0',
                                          'κανένα':'0',
                                          "έναν": "1", 
                                          "δύο": "2", 
                                          'τρεις':'3', 
                                          'τέσσερις':'4',
                                         'πέντε':'5',
                                         'έξι':'6',
                                         'εφτά':'7',
                                         'επτά':'7',
                                         'οχτώ':'8',
                                         'οκτώ':'8',
                                         'εννέα':'9',
                                         'ενιά':'9',
                                         'δέκα':'10',
                                         'έντεκα':'11',
                                         'δώδεκα':'12'}).astype(float)

In [9]:
df['new_cases'] = df.new_cases.astype(float)

In [10]:
df['total_cases'] = df.content.str.extract(r'συνολικός\s+αριθμός.*κρουσμάτων\s+είναι\s+(\d+[.]?\d+)',re.IGNORECASE)
df['total_cases'] = df.total_cases.str.replace('.','').astype(float)

In [11]:
df['new_deaths'] = df.content.str.extract(r'(?:μεθ.*\w+με\s+|μεθ.*\w+με\s+.*\s+)(\w+)(?:\s+καταγεγραμμένο.*θ[άα]ν[αά]το.*\s+και.*στη\s+χώρα.*|\s+ακόμα\s+καταγεγραμμένο.*θ[άα]ν[αά]το.*\s+και.*στη\s+χώρα.*|\s+νέο.*θ[άα]ν[αά]το.*και.*στη\s+χώρα.*|\s+θ[άα]ν[αά]το.*και.*στη\s+χώρα.*)',re.IGNORECASE)

In [12]:
df['new_deaths'] = df.new_deaths.replace({'μηδέν':'0',
                                          'κανέναν':'0',
                                          'κανένα':'0',
                                          "έναν": "1", 
                                          "δύο": "2", 
                                          'τρεις':'3', 
                                          'τέσσερις':'4',
                                         'πέντε':'5',
                                         'έξι':'6',
                                         'εφτά':'7',
                                         'επτά':'7',
                                         'οχτώ':'8',
                                         'οκτώ':'8',
                                         'εννέα':'9',
                                         'ενιά':'9',
                                         'δέκα':'10',
                                         'έντεκα':'11',
                                         'δώδεκα':'12'}).astype(float)

In [13]:
df['total_deaths'] = df.content.str.extract(r'θ[άα]ν[αά]το[υς ]{,3}.*?([0-9]{3})\s*θ[άα]ν[αά]το[υς ]{,3}.*?στη\s+χώρα',re.IGNORECASE).astype(float)

In [14]:
df['intubated'] = df.content.str.extract(r'(\d+)\s+συμπολίτες\s+μας\s+νοσηλεύονται\s+διασωληνωμένοι').astype(float)

In [15]:
df['total_tests'] = df.content.str.extract(r'ελεγχθεί.*(\d+|\d+[.]\d+)\s+κλινικά\s+δείγματα',re.IGNORECASE)
df['total_tests'] = df.total_tests.str.replace('.','').astype(float)

In [16]:
#current total number of cases, according to content scraped/data extracted
totalcases_s = df.total_cases[0]
#current number of new cases, according to content scraped/data extracted
newcases_s = df.new_cases[0]

#current total number of deaths, according to content scraped/data extracted
totaldeaths_s = df.total_deaths[0]
#current number of new deaths, according to content scraped/data extracted
newdeaths_s = df.new_deaths[0]

#current total number of intubated patiens, according to content scraped/data extracted
intub_s = df.intubated[0]

#current total number of tests, according to content scraped/data extracted
totaltests_s = df['total_tests'][0]

#the date of latest official announcement scraped
date_s = df['date'][0]

#today
today = datetime.today().strftime("%Y-%m-%d")

In [17]:
greece = pd.read_csv('https://raw.githubusercontent.com/iMEdD-Lab/open-data/master/COVID-19/greece.csv')
# ('https://docs.google.com/spreadsheets/d/e/2PACX-1vRpR8AOJaRsB5by7H3R_GijtaY06J8srELipebO5B0jYEg9pKugT3C6Rk2RSQ5eyerQl7LolshamK27/pub?gid=1017539712&single=true&output=csv')

#total number of cases in our current dataset
greececasessum = greece.cases.sum()
#number of cases for which location remains unknown in our current dataset
greececasesunknown = greece[greece['county_en'] == 'Unknown']['cases']

#total number of deaths in our current dataset
greecedeadsum = greece.dead.sum()
#number of deaths for which location remains unknown in our current dataset
greecedeadunknown = greece[greece['county_en'] == 'Unknown']['dead']

#total number of intubated patients in our current dataset
greeceintubsum = greece.critical.sum()
#number of intubated patients whose region of origin remains unknown in our current dataset
greeceintubunknown = greece[greece['county_en'] == 'Unknown']['critical']

In [18]:
if date_s == today:

    if greececasessum < totalcases_s and greececasessum+newcases_s == totalcases_s:
        greececasesunknown = greececasesunknown+(totalcases_s-greececasessum)
        greece.loc[greece['county_en'] == 'Unknown', 'cases'] = greececasesunknown

        greececases = 'greece.cases was updated and stored.'

    elif greececasessum < totalcases_s and greececasessum+newcases_s != totalcases_s:
        greececases = 'Be careful: Something goes wrong with cases announced!!!'

    elif greececasessum >= totalcases_s:
        greececases = 'Cool: greece.cases is updated!'

    else:
        greececases = 'Sorry, I do not know what is happening with greece.cases!'



    if greecedeadsum < totaldeaths_s: 
    # and greecedeadsum+newdeaths_s == totaldeaths_s:
        greecedeadunknown = greecedeadunknown+(totaldeaths_s-greecedeadsum)
        greece.loc[greece['county_en'] == 'Unknown', 'dead'] = greecedeadunknown

        greecedead = 'greece.dead was updated and stored.'

    # elif greecedeadsum < totaldeaths_s and greecedeadsum+newdeaths_s != totaldeaths_s:
    #     greecedead = 'Be careful: Something goes wrong with deaths announced!!!'

    elif greecedeadsum >= totaldeaths_s:
        greecedead = 'Cool: greece.dead is updated!'

    else:
        greecedead = 'Sorry, I do not know what is happening with greece.dead!'



    if greeceintubsum != intub_s and greeceintubsum == greeceintubunknown.tolist()[0]:
        greeceintubunknown = intub_s
        greece.loc[greece['county_en'] == 'Unknown', 'critical'] = greeceintubunknown

        greeceintub = 'greece.critical was updated and stored.'

    elif greeceintubsum == intub_s:
        greeceintub = 'Cool: greece.critical is updated!'

    else:
        greeceintub = 'Sorry, I do not know what is happening with greece.critical!'


    greece.to_csv('greece_autobriefing_test.csv',index=False)
    # greece.to_csv(f'/root/AUTO_GIT/open-data/COVID-19/greece.csv',index=False, encoding = 'utf-8')
    # time = time.strftime("%Y_%m_%d-%H_%M")
    # greece.to_csv(f'greece_auto_test{time}.csv',index=False, encoding = 'utf-8')
    # time.strftime("%Y_%m_%d-%H_%M")}.csv',index=False, encoding = 'utf-8')


    if greececases == 'greece.cases was updated and stored.' or greecedead == 'greece.dead was updated and stored.' or greeceintub == 'greece.critical was updated and stored.':

        greecetext = greececases+' '+greecedead+' '+greeceintub+' '+'greece_auto_test.csv has been saved.' 

    else: 

        greecetext = greececases+' '+greecedead+' '+greeceintub

    
else:
    greecetext = 'Latest announcement published refers to '+str(date_s)+'. So, greece.csv should be updated.'    


In [19]:
greecetext

'Latest announcement published refers to 2020-05-24. So, greece.csv should be updated.'

In [20]:
greeceTimeline = pd.read_csv('https://raw.githubusercontent.com/iMEdD-Lab/open-data/master/COVID-19/greeceTimeline.csv')
# ('https://raw.githubusercontent.com/iMEdD-Lab/open-data/master/COVID-19/greeceTimeline.csv')
# greeceTimeline = greeceTimeline.drop(greeceTimeline.columns[-1],axis=1)
grt = greeceTimeline
grt = grt.drop(["Province/State", "Country/Region"], axis=1)
grt = grt.set_index("Status")
grt = grt.T
grt = grt.reset_index()
grt = grt.rename(columns={"index": "date"})
grt['date'] = pd.to_datetime(grt['date']).dt.strftime("%Y-%m-%d")

last_date = grt['date'].iloc[-1]

In [21]:
timeline = {}
# if grt['date'].iloc[-1] < df['date'][0]:
if last_date < today:
# and date_s == today:

    timeline['date'] = date_s
    timeline['cases'] = newcases_s
    timeline['total cases'] = totalcases_s
    timeline['deaths'] = newdeaths_s
    timeline['intubated'] = intub_s
    timeline['total_tests'] = totaltests_s
     
    grt = grt.append(timeline,ignore_index=True)
    
    grt['date'] = grt.date.astype(str)
    grt['year'] = grt.date.str.extract(r'(\d\d)\d\d-\d\d-\d\d')
    grt['month'] = grt.date.str.extract(r'\d\d\d\d-(\d\d)-\d\d')
    grt['month'] = grt.month.replace({'01':'1',
                                   '02':'2',
                                   '03':'3',
                                   '04':'4',
                                   '05':'5',
                                   '06':'6',
                                   '07':'7',
                                   '08':'8',
                                   '09':'9'})
    grt['day'] = grt.date.str.extract(r'\d\d\d\d-\d\d-(\d\d)')
    grt['day'] = grt.day.replace({'01':'1',
                               '02':'2',
                               '03':'3',
                               '04':'4',
                               '05':'5',
                               '06':'6',
                               '07':'7',
                               '08':'8',
                               '09':'9'})
    grt['date2'] = grt.month+'/'+grt.day+'/'+grt.year
    grt = grt.drop(['date','year','month','day'],axis=1)
    
    grt = grt.set_index('date2').T.reset_index()
    grt = grt[['Status',grt.columns[-1]]].copy()
    grt_new = pd.merge(greeceTimeline,grt,on='Status',how='outer')

    grt_new.to_csv('greeceTimeline_autobriefing_test.csv',index=False)
#     grt_new.to_csv(f'/root/AUTO_GIT/open-data/COVID-19/greeceTimeline.csv',index=False, encoding = 'utf-8')
#     grt_new.to_csv(f'AUTO_GIT/AUTO_COVID_DATA/greeceTimeline_autobriefing_test{time.strftime("%Y_%m_%d-%H_%M")}.csv',index=False, encoding = 'utf-8')

    
    greeceTimeline_text = 'Daily update: greeceTimeline appended and greeceTimeline_autobriefing_test.csv has been saved.'

elif last_date < today and date_s != today:
    
    greeceTimeline_text = 'greece.Timeline is not updated, because no announcement has been out for today. Please wait or check out what is happening!'
    
elif last_date == today:
    
    greeceTimeline_text = 'Cool: greece.Timeline is updated!'

else: 
    greeceTimeline_text = 'Sorry, I do not know what is happening with greece.Timeline!'
